# 1. Importar librerías

In [129]:
#Librerías para manipulación de datos
import pandas as pd
import numpy as np

#Librerías para visualización
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Importar módulo de funciones
import sys
sys.path.insert(0, 'utils')  # Añadir el directorio 'utils' al PATH
import funciones
from importlib import reload
reload(funciones)  # Recarga el módulo

<module 'funciones' from 'c:\\recursos_humanos\\utils\\funciones.py'>

# 2. Importar datos

In [130]:
data = funciones.get_data('data_train')

In [131]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   InfoDate                 4410 non-null   dbdate 
 1   EmployeeID               4410 non-null   Int64  
 2   Age                      4410 non-null   Int64  
 3   DistanceFromHome         4410 non-null   Int64  
 4   MonthlyIncome            4410 non-null   Int64  
 5   NumCompaniesWorked       4410 non-null   float64
 6   PercentSalaryHike        4410 non-null   Int64  
 7   StockOptionLevel         4410 non-null   Int64  
 8   TrainingTimesLastYear    4410 non-null   Int64  
 9   YearsAtCompany           4410 non-null   Int64  
 10  YearsSinceLastPromotion  4410 non-null   Int64  
 11  YearsWithCurrManager     4410 non-null   Int64  
 12  TotalWorkingYears        4410 non-null   float64
 13  EnvironmentSatisfaction  4410 non-null   float64
 14  JobSatisfaction         

# 3. Análisis de correlaciones

In [132]:
numeric_cols = ['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked',
                'PercentSalaryHike', 'TrainingTimesLastYear', 'YearsAtCompany',
                'YearsSinceLastPromotion', 'YearsWithCurrManager', 'TotalWorkingYears']

In [133]:
corr_matrix = data[numeric_cols].corr()
corr_matrix = corr_matrix.apply(lambda x: np.round(x, 3))
fig = px.imshow(corr_matrix, text_auto=True)
fig.layout.update(height=700)
fig.show()

La variable **años trabajando en la compañía** presenta una alta correlación con las variables **total de años trabajando**, **años desde la última promoción** y **años con el jefe actual**. Además, la edad también presenta una alta correlación con la variable **total de años trabajando**.

Por lo anterior, se decide descartar la variable **Total de años trabajando** por presentar alta correlación con 2 variables y la variable **años con el jefe actual**. Además, se descarta también **años desde la última promoción**, que ya vimos en el análisis exploratorio que no tiene una influencia clara en el retiro de los colaboradores.

In [134]:
data_no_corr = data.drop(['YearsSinceLastPromotion', 'YearsWithCurrManager', 'TotalWorkingYears'], axis = 1)

# 4. Tratamiento de datos atípicos

## 4.1. Visualización de valores atípicos

In [135]:
numeric_cols_no_corr = ['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked',
                'PercentSalaryHike', 'TrainingTimesLastYear', 'YearsAtCompany']

In [136]:
for col in numeric_cols_no_corr:
    funciones.vizualizar_outliers(data_no_corr, col)

Analicemos si estos datos atípicos son un comportamiento natural de los datos antes de imputarlos.

## 4.2. Análisis de valores atípicos en los ingresos

### ¿Los ingresos dependen del nivel del puesto, nivel de estudios, rol o del departamento?

In [137]:
for col in ['JobLevel', 'Education', 'JobRole', 'Department']:
    fig = px.box(data, y = 'MonthlyIncome', x = col, color = col, title= 'Ingresos mensuales por '+col)
    fig.show()

### ¿A más tiempo en la empresa tiende a ser mayor los ingresos mensuales?

In [138]:
px.scatter(data, x='YearsAtCompany', y='MonthlyIncome', title='Ingresos mensuales vs Años en la compañía')

Todas las distribuciones presentan valores atípicos en sus ingresos y no hay una tendencia o correlación que indique que a más años en la compañía mayor serán los ingresos, permitiendo concluir que los ingresos altos no dependen de ninguna de estas variables. Dicho esto y que los atípicos en las otras variables también pueden considerarse naturales, es posible imputar valores atípicos pero utilizando un factor de ampliación alto (3.0) para no afectar significativamente la naturalidad de los datos.

## 4.3. Imputación de valores (técnica con IQR)

Se usa un factor de ampliación de 3.0, para solo imputar los valores demasiado extremos.

In [145]:
cols_with_outliers = ['MonthlyIncome', 'NumCompaniesWorked', 'TrainingTimesLastYear', 'YearsAtCompany']
data_without_outliers = funciones.imputar_outliers(data_no_corr, cols_with_outliers, 3)
data_without_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4353 entries, 0 to 4352
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   InfoDate                 4353 non-null   dbdate 
 1   EmployeeID               4353 non-null   Int64  
 2   Age                      4353 non-null   Int64  
 3   DistanceFromHome         4353 non-null   Int64  
 4   MonthlyIncome            4353 non-null   Int64  
 5   NumCompaniesWorked       4353 non-null   float64
 6   PercentSalaryHike        4353 non-null   Int64  
 7   StockOptionLevel         4353 non-null   Int64  
 8   TrainingTimesLastYear    4353 non-null   Int64  
 9   YearsAtCompany           4353 non-null   Int64  
 10  EnvironmentSatisfaction  4353 non-null   float64
 11  JobSatisfaction          4353 non-null   float64
 12  WorkLifeBalance          4353 non-null   float64
 13  BusinessTravel           4353 non-null   object 
 14  Department               4353

# 5. Transformaciones de variables

In [149]:
data2 = data_without_outliers.copy()
data2['StockOptionLevel'] = data2['StockOptionLevel'].astype('category')
data2['EnvironmentSatisfaction'] = data2['EnvironmentSatisfaction'].astype('int64')
data2['JobSatisfaction'] = data2['JobSatisfaction'].astype('int64')
data2['WorkLifeBalance'] = data2['StockOptionLevel'].astype('int64')

In [151]:
data2.to_csv('data/data_train_clean.csv', index = False)